In [31]:
import pandas as pd

# Load datasets
df = pd.read_csv("output/groundwater_filtered_clean.csv")
baseline = pd.read_csv("output/station_baseline.csv")

# Parse date
df['DATE'] = pd.to_datetime(df['DATE'])
df['year'] = df['DATE'].dt.year

print("Records:", len(df))
print("Stations:", df['station_id'].nunique())


Records: 86912
Stations: 9632


In [32]:
df_gavi = df.merge(
    baseline[['station_id', 'min_wl', 'max_wl']],
    on='station_id',
    how='inner'
)

print("After merge records:", len(df_gavi))


After merge records: 86515


In [33]:
df_gavi['GAVI'] = 100 * (
    1 - (df_gavi['WL_MBGL'] - df_gavi['min_wl']) /
        (df_gavi['max_wl'] - df_gavi['min_wl'])
)

# Clip to safe bounds
df_gavi['GAVI'] = df_gavi['GAVI'].clip(0, 100)


In [34]:
def gavi_category(g):
    if g >= 75:
        return "Safe"
    elif g >= 50:
        return "Watch"
    elif g >= 25:
        return "Stressed"
    else:
        return "Critical"

df_gavi['GAVI_CATEGORY'] = df_gavi['GAVI'].apply(gavi_category)


In [35]:
print(df_gavi['GAVI'].describe())
print(df_gavi['GAVI_CATEGORY'].value_counts(normalize=True) * 100)


count    86515.000000
mean        52.811472
std         34.341167
min          0.000000
25%         22.222222
50%         55.454545
75%         84.210526
max        100.000000
Name: GAVI, dtype: float64
GAVI_CATEGORY
Safe        33.507484
Critical    26.863550
Watch       21.278391
Stressed    18.350575
Name: proportion, dtype: float64


In [36]:
df_gavi.to_csv(
    "output/groundwater_gavi_2015_2024.csv",
    index=False
)

print("✓ GAVI dataset saved successfully!")


✓ GAVI dataset saved successfully!


## GAVI-Based Groundwater Alert System

Implement a multi-layered alert system with 4 alert types:
- 🔴 Critical Groundwater (GAVI < 25)
- 🟠 Depletion Warning (GAVI < 50 AND delta_wl < 0)
- 🟡 Sudden Drop (delta_wl ≤ -2.0)
- 🟢 Recovery Signal (delta_wl ≥ +1.0)

In [37]:
# Define master alert function with priority hierarchy
def generate_alert(row):
    """
    Priority-based alert system:
    1. Critical Groundwater (GAVI < 25)
    2. Depletion Warning (GAVI < 50 AND delta_wl < 0)
    3. Sudden Drop (delta_wl ≤ -2.0)
    4. Recovery Signal (delta_wl ≥ +1.0)
    5. Normal (default)
    """
    if row['GAVI'] < 25:
        return "CRITICAL_GROUNDWATER"
    if row['GAVI'] < 50 and row['delta_wl'] < 0:
        return "DEPLETION_WARNING"
    if row['delta_wl'] <= -2.0:
        return "SUDDEN_DROP"
    if row['delta_wl'] >= 1.0:
        return "RECOVERY_SIGNAL"
    return "NORMAL"

# Apply alert function
df_gavi['ALERT'] = df_gavi.apply(generate_alert, axis=1)

print("✓ Alerts generated successfully!")
print(f"\nTotal records: {len(df_gavi)}")
print(f"\nAlert distribution:")
print(df_gavi['ALERT'].value_counts())
print(f"\nAlert percentage:")
print((df_gavi['ALERT'].value_counts(normalize=True) * 100).round(2))

✓ Alerts generated successfully!

Total records: 86515

Alert distribution:
ALERT
NORMAL                  40798
CRITICAL_GROUNDWATER    23241
SUDDEN_DROP              8884
RECOVERY_SIGNAL          7909
DEPLETION_WARNING        5683
Name: count, dtype: int64

Alert percentage:
ALERT
NORMAL                  47.16
CRITICAL_GROUNDWATER    26.86
SUDDEN_DROP             10.27
RECOVERY_SIGNAL          9.14
DEPLETION_WARNING        6.57
Name: proportion, dtype: float64


In [38]:
# Add stability/persistence check for Critical alerts
# Sort by station and date
df_gavi = df_gavi.sort_values(['station_id', 'DATE'])

# Calculate persistence: number of consecutive critical conditions
df_gavi['critical_flag'] = (df_gavi['GAVI'] < 25).astype(int)

# Rolling sum over 3 observations per station
df_gavi['critical_persist'] = (
    df_gavi.groupby('station_id')['critical_flag']
    .rolling(window=3, min_periods=1)
    .sum()
    .reset_index(level=0, drop=True)
)

# Add confirmed alert flag (requires 2+ consecutive observations)
df_gavi['ALERT_CONFIRMED'] = df_gavi['ALERT']
df_gavi.loc[
    (df_gavi['ALERT'] == 'CRITICAL_GROUNDWATER') & (df_gavi['critical_persist'] < 2),
    'ALERT_CONFIRMED'
] = 'NORMAL'

print("✓ Persistence checks applied!")
print(f"\nCritical alerts before persistence check: {(df_gavi['ALERT'] == 'CRITICAL_GROUNDWATER').sum()}")
print(f"Critical alerts after persistence check: {(df_gavi['ALERT_CONFIRMED'] == 'CRITICAL_GROUNDWATER').sum()}")
print(f"\nConfirmed alert distribution:")
print(df_gavi['ALERT_CONFIRMED'].value_counts())

✓ Persistence checks applied!

Critical alerts before persistence check: 23241
Critical alerts after persistence check: 10548

Confirmed alert distribution:
ALERT_CONFIRMED
NORMAL                  53491
CRITICAL_GROUNDWATER    10548
SUDDEN_DROP              8884
RECOVERY_SIGNAL          7909
DEPLETION_WARNING        5683
Name: count, dtype: int64


In [39]:
# Alert → Action mapping
alert_actions = {
    'CRITICAL_GROUNDWATER': 'Extraction restriction, emergency planning',
    'DEPLETION_WARNING': 'Monitoring, demand regulation',
    'SUDDEN_DROP': 'Field verification required',
    'RECOVERY_SIGNAL': 'Recharge success validation',
    'NORMAL': 'Continue regular monitoring'
}

# Add action column
df_gavi['SUGGESTED_ACTION'] = df_gavi['ALERT_CONFIRMED'].map(alert_actions)

# Add alert severity
alert_severity = {
    'CRITICAL_GROUNDWATER': 'CRITICAL',
    'DEPLETION_WARNING': 'HIGH',
    'SUDDEN_DROP': 'MEDIUM',
    'RECOVERY_SIGNAL': 'POSITIVE',
    'NORMAL': 'NORMAL'
}

df_gavi['ALERT_SEVERITY'] = df_gavi['ALERT_CONFIRMED'].map(alert_severity)

print("✓ Alert actions and severity added!")
print("\nSample alerts with actions:")
print(df_gavi[df_gavi['ALERT_CONFIRMED'] != 'NORMAL'][
    ['station_id', 'DATE', 'GAVI', 'ALERT_CONFIRMED', 'ALERT_SEVERITY', 'SUGGESTED_ACTION']
].head(10))

✓ Alert actions and severity added!

Sample alerts with actions:
                        station_id       DATE        GAVI  \
1    Andhra Pradesh_13.1956_79.004 2016-01-01   92.452830   
2    Andhra Pradesh_13.1956_79.004 2017-01-01    1.886792   
3    Andhra Pradesh_13.1956_79.004 2018-01-01   77.358491   
5    Andhra Pradesh_13.1956_79.004 2023-01-01   56.603774   
8   Andhra Pradesh_13.2075_79.0981 2016-01-01   88.524590   
9   Andhra Pradesh_13.2075_79.0981 2017-01-01   60.655738   
13  Andhra Pradesh_13.2075_79.0981 2024-01-01   49.180328   
15  Andhra Pradesh_13.4262_79.4531 2016-01-01  100.000000   
16  Andhra Pradesh_13.4262_79.4531 2017-01-01   60.000000   
20  Andhra Pradesh_13.4262_79.4531 2024-01-01   69.411765   

         ALERT_CONFIRMED ALERT_SEVERITY  \
1            SUDDEN_DROP         MEDIUM   
2   CRITICAL_GROUNDWATER       CRITICAL   
3            SUDDEN_DROP         MEDIUM   
5        RECOVERY_SIGNAL       POSITIVE   
8            SUDDEN_DROP         MEDIUM   
9    

## District and State Level Alert Aggregation

Aggregate alerts to district and state level for policy decision-making.

In [40]:
# District-level alert aggregation
district_alerts = (
    df_gavi[df_gavi['ALERT_CONFIRMED'] != "NORMAL"]
    .groupby(['STATE_UT', 'DISTRICT', 'ALERT_CONFIRMED'])
    .size()
    .reset_index(name='alert_count')
)

print("District-level alert summary:")
print(district_alerts.head(20))
print("\n" + "="*70)

# CORRECTED: Compute station-level stress (any observation with GAVI < 50)
# Stressed OR Critical only (GAVI < 50)
stress_mask = df_gavi['GAVI_CATEGORY'].isin(['Stressed', 'Critical'])

# For each station, check if it has ANY stressed observation
station_has_stress = stress_mask.groupby(df_gavi['station_id']).any()

# Map back to main dataframe
df_gavi['is_stressed_station'] = df_gavi['station_id'].map(station_has_stress)

# Calculate stressed stations ratio by district
district_stress = df_gavi.groupby(['STATE_UT', 'DISTRICT']).agg(
    total_stations=('station_id', 'nunique'),
    total_records=('station_id', 'count'),
    critical_alerts=('ALERT_CONFIRMED', lambda x: (x == 'CRITICAL_GROUNDWATER').sum()),
    depletion_alerts=('ALERT_CONFIRMED', lambda x: (x == 'DEPLETION_WARNING').sum()),
    avg_gavi=('GAVI', 'mean'),
    stressed_ratio=('is_stressed_station', lambda x: (x.groupby(df_gavi.loc[x.index, 'station_id']).first().mean() * 100))
).reset_index()

district_stress = district_stress.sort_values('stressed_ratio', ascending=False)

print("\nTop 10 most stressed districts:")
print(district_stress.head(10))
print("\n" + "="*70)

# State-level aggregation
state_summary = df_gavi.groupby('STATE_UT').agg(
    total_stations=('station_id', 'nunique'),
    total_records=('station_id', 'count'),
    avg_gavi=('GAVI', 'mean'),
    critical_count=('ALERT_CONFIRMED', lambda x: (x == 'CRITICAL_GROUNDWATER').sum()),
    depletion_count=('ALERT_CONFIRMED', lambda x: (x == 'DEPLETION_WARNING').sum()),
    recovery_count=('ALERT_CONFIRMED', lambda x: (x == 'RECOVERY_SIGNAL').sum()),
    stressed_pct=('is_stressed_station', lambda x: (x.groupby(df_gavi.loc[x.index, 'station_id']).first().mean() * 100))
).reset_index()

state_summary = state_summary.sort_values('stressed_pct', ascending=False)

print("\nState-level alert summary:")
print(state_summary)

District-level alert summary:
          STATE_UT               DISTRICT       ALERT_CONFIRMED  alert_count
0   Andhra Pradesh  Alluri Sitharama Raju  CRITICAL_GROUNDWATER           14
1   Andhra Pradesh  Alluri Sitharama Raju     DEPLETION_WARNING            8
2   Andhra Pradesh  Alluri Sitharama Raju       RECOVERY_SIGNAL            6
3   Andhra Pradesh  Alluri Sitharama Raju           SUDDEN_DROP            3
4   Andhra Pradesh             Anakapalli  CRITICAL_GROUNDWATER            4
5   Andhra Pradesh             Anakapalli     DEPLETION_WARNING            8
6   Andhra Pradesh             Anakapalli       RECOVERY_SIGNAL            5
7   Andhra Pradesh             Anakapalli           SUDDEN_DROP            2
8   Andhra Pradesh          Ananthapuramu  CRITICAL_GROUNDWATER            7
9   Andhra Pradesh          Ananthapuramu     DEPLETION_WARNING            2
10  Andhra Pradesh          Ananthapuramu       RECOVERY_SIGNAL            6
11  Andhra Pradesh          Ananthapuramu     

In [62]:
# CURRENT SNAPSHOT FIX: Create latest observation per station for district map
print("="*70)
print("CREATING CURRENT GROUNDWATER SNAPSHOT")
print("="*70)

# Get latest observation per station (most recent year)
latest_station_snapshot = (
    df_gavi.sort_values('year')
    .groupby('station_id')
    .tail(1)
    .reset_index(drop=True)
)

# Define current stress flag (GAVI < 50 in latest observation)
latest_station_snapshot['is_stressed_current'] = (latest_station_snapshot['GAVI'] < 50)

print(f"\n✓ Latest snapshot created: {len(latest_station_snapshot)} stations")
print(f"✓ Most recent year: {latest_station_snapshot['year'].max()}")
print(f"\nCurrent stress distribution:")
print(f"  Stressed (GAVI < 50): {latest_station_snapshot['is_stressed_current'].sum()} stations")
print(f"  Safe (GAVI >= 50): {(~latest_station_snapshot['is_stressed_current']).sum()} stations")
print(f"  Stressed %: {latest_station_snapshot['is_stressed_current'].mean() * 100:.1f}%")

# RECOMPUTE district stress using CURRENT snapshot only
district_stress_current = latest_station_snapshot.groupby(['STATE_UT', 'DISTRICT']).agg(
    total_stations=('station_id', 'nunique'),
    stressed_stations=('is_stressed_current', 'sum'),
    avg_gavi=('GAVI', 'mean'),
    critical_alerts=('ALERT_CONFIRMED', lambda x: (x == 'CRITICAL_GROUNDWATER').sum()),
    depletion_alerts=('ALERT_CONFIRMED', lambda x: (x == 'DEPLETION_WARNING').sum())
).reset_index()

# Calculate stressed ratio
district_stress_current['stressed_ratio'] = (
    district_stress_current['stressed_stations'] / district_stress_current['total_stations'] * 100
)

district_stress_current = district_stress_current.sort_values('stressed_ratio', ascending=False)

print(f"\n✓ District stress (CURRENT) computed for {len(district_stress_current)} districts")
print(f"\nTop 10 most stressed districts (CURRENT snapshot):")
print(district_stress_current[['STATE_UT', 'DISTRICT', 'stressed_ratio', 'avg_gavi', 'critical_alerts']].head(10))
print("\n" + "="*70)

CREATING CURRENT GROUNDWATER SNAPSHOT

✓ Latest snapshot created: 9545 stations
✓ Most recent year: 2024

Current stress distribution:
  Stressed (GAVI < 50): 4133 stations
  Safe (GAVI >= 50): 5412 stations
  Stressed %: 43.3%

✓ District stress (CURRENT) computed for 713 districts

Top 10 most stressed districts (CURRENT snapshot):
          STATE_UT         DISTRICT  stressed_ratio   avg_gavi  \
661  Uttar Pradesh       Mahrajganj           100.0  31.641124   
672  Uttar Pradesh           Rampur           100.0   6.818182   
113          Bihar       Sheikhpura           100.0   9.438776   
115          Bihar        Sitamarhi           100.0   8.056634   
114          Bihar          Sheohar           100.0   0.000000   
611        Tripura     West Tripura           100.0  32.367632   
116          Bihar            Siwan           100.0  15.403245   
379    Maharashtra  DharashivKalamb           100.0  47.368421   
664  Uttar Pradesh              Mau           100.0  28.533651   
412 

In [42]:
# Save all alert datasets
print("="*70)
print("SAVING ALERT DATASETS")
print("="*70)

# 1. Save main GAVI + Alerts dataset
gavi_alerts_file = 'output/groundwater_gavi_alerts_2015_2024.csv'
df_gavi.to_csv(gavi_alerts_file, index=False)
print(f"\n✓ Saved: {gavi_alerts_file}")

# 2. Save district-level alerts
district_alerts_file = 'output/district_level_alerts.csv'
district_alerts.to_csv(district_alerts_file, index=False)
print(f"✓ Saved: {district_alerts_file}")

# 3. Save district stress summary
district_stress_file = 'output/district_stress_summary.csv'
district_stress.to_csv(district_stress_file, index=False)
print(f"✓ Saved: {district_stress_file}")

# 4. Save state summary
state_summary_file = 'output/state_alert_summary.csv'
state_summary.to_csv(state_summary_file, index=False)
print(f"✓ Saved: {state_summary_file}")

print("\n" + "="*70)
print("✓ ALL ALERT DATASETS SAVED SUCCESSFULLY!")
print("="*70)

print("\nFiles created:")
print(f"  1. {gavi_alerts_file}")
print(f"  2. {district_alerts_file}")
print(f"  3. {district_stress_file}")
print(f"  4. {state_summary_file}")

SAVING ALERT DATASETS

✓ Saved: output/groundwater_gavi_alerts_2015_2024.csv
✓ Saved: output/district_level_alerts.csv
✓ Saved: output/district_stress_summary.csv
✓ Saved: output/state_alert_summary.csv

✓ ALL ALERT DATASETS SAVED SUCCESSFULLY!

Files created:
  1. output/groundwater_gavi_alerts_2015_2024.csv
  2. output/district_level_alerts.csv
  3. output/district_stress_summary.csv
  4. output/state_alert_summary.csv


In [61]:
# Generate policy-ready statements
print("="*70)
print("POLICY-READY ALERT STATEMENTS")
print("="*70)

# Overall statistics (using CURRENT snapshot: latest observation per station)
total_stations = latest_station_snapshot['station_id'].nunique()
# Count stations with CURRENT stressed condition (GAVI < 50 in latest observation)
stressed_stations = (
    latest_station_snapshot[latest_station_snapshot['GAVI'] < 50]
    ['station_id']
    .nunique()
)
stressed_pct = (stressed_stations / total_stations) * 100

print(f"\n📊 OVERALL ASSESSMENT:")
print(f"   Total monitoring stations: {total_stations}")
print(f"   Stations under stress (GAVI < 50): {stressed_stations} ({stressed_pct:.1f}%)")
print(f"   Average GAVI: {latest_station_snapshot['GAVI'].mean():.1f}")

# Critical districts (using CURRENT snapshot)
critical_districts = district_stress_current[district_stress_current['critical_alerts'] > 0].head(3)
if len(critical_districts) > 0:
    print(f"\n🔴 TOP CRITICAL DISTRICTS:")
    for idx, row in critical_districts.iterrows():
        print(f"   • {row['DISTRICT']}, {row['STATE_UT']}")
        print(f"     - {row['critical_alerts']} critical alerts")
        print(f"     - {row['stressed_ratio']:.1f}% stations under stress")
        print(f"     - Suggested action: Immediate extraction restriction and emergency planning")

# Recovery signals
recovery_count = (df_gavi['ALERT_CONFIRMED'] == 'RECOVERY_SIGNAL').sum()
if recovery_count > 0:
    print(f"\n🟢 POSITIVE SIGNALS:")
    print(f"   {recovery_count} recovery/recharge signals detected")
    print(f"   Indicates successful monsoon recharge or reduced extraction")

print("="*70)

print("\n" + "="*70)
print("✓ ALERT SYSTEM COMPLETE AND READY FOR DEPLOYMENT!")

POLICY-READY ALERT STATEMENTS

📊 OVERALL ASSESSMENT:
   Total monitoring stations: 9545
   Stations under stress (GAVI < 50): 4133 (43.3%)
   Average GAVI: 54.1

🔴 TOP CRITICAL DISTRICTS:
   • Mahrajganj, Uttar Pradesh
     - 1 critical alerts
     - 100.0% stations under stress
     - Suggested action: Immediate extraction restriction and emergency planning
   • Rampur, Uttar Pradesh
     - 1 critical alerts
     - 100.0% stations under stress
     - Suggested action: Immediate extraction restriction and emergency planning
   • Sheikhpura, Bihar
     - 2 critical alerts
     - 100.0% stations under stress
     - Suggested action: Immediate extraction restriction and emergency planning

🟢 POSITIVE SIGNALS:
   7909 recovery/recharge signals detected
   Indicates successful monsoon recharge or reduced extraction

✓ ALERT SYSTEM COMPLETE AND READY FOR DEPLOYMENT!


# SECTION 5 — AI / ANALYTICS: PREDICTIVE FORECASTING

## Goal: From "current status" → "what happens next"

**Flow:**
1. Clean WL time series ✅
2. Baseline trend model (mandatory)
3. ML forecast model (optional, representative stations)
4. Forecasted WL → Future GAVI → Future alerts

**Decision locked:** Annual WL forecasting (2015-2024 data)

## STEP 5.2 — PREPARE MODELING DATASET

Create model-ready dataframe with clean time series:
- Remove first year per station (delta_wl = NaN)
- Sort by station + year
- Ensure no WL > 50
- Minimum years per station ≥ 5

In [64]:
# STEP 5.2: Prepare modeling dataset
df_model = df_gavi.copy()

# Remove records with missing delta_wl (first year per station)
df_model = df_model.dropna(subset=['delta_wl'])

# Sort by station and year
df_model = df_model.sort_values(['station_id', 'year'])

# Filter stations with at least 5 years of data
station_years = df_model.groupby('station_id')['year'].count()
valid_stations = station_years[station_years >= 5].index
df_model = df_model[df_model['station_id'].isin(valid_stations)]

print("="*70)
print("MODELING DATASET PREPARED")
print("="*70)
print(f"\n✓ Total records: {len(df_model)}")
print(f"✓ Total stations: {df_model['station_id'].nunique()}")
print(f"✓ Year range: {df_model['year'].min()} - {df_model['year'].max()}")
print(f"✓ WL range: {df_model['WL_MBGL'].min():.2f} - {df_model['WL_MBGL'].max():.2f} mbgl")
print(f"\nRecords per station (stats):")
print(df_model.groupby('station_id').size().describe())
print("\n" + "="*70)

MODELING DATASET PREPARED

✓ Total records: 76970
✓ Total stations: 9545
✓ Year range: 2015 - 2024
✓ WL range: 0.00 - 50.00 mbgl

Records per station (stats):
count    9545.000000
mean        8.063908
std         1.714877
min         5.000000
25%         7.000000
50%         8.000000
75%         9.000000
max        36.000000
dtype: float64



## STEP 5.3 — BASELINE FORECAST MODEL (MANDATORY)

**Model Logic:** Future WL = last WL + mean historical rate of change

This is the fallback + benchmark model that uses historical trends to predict future water levels.

In [45]:
# STEP 5.3: Baseline forecast model

# Calculate mean delta_wl per station
station_trend = (
    df_model.groupby('station_id')['delta_wl']
    .mean()
    .reset_index(name='mean_delta')
)

# Get latest WL for each station
latest_wl = (
    df_model.sort_values('year')
    .groupby('station_id')
    .tail(1)
    [['station_id', 'year', 'WL_MBGL', 'GAVI', 'STATE_UT', 'DISTRICT']]
    .reset_index(drop=True)
)

# Merge with trend
forecast_base = latest_wl.merge(station_trend, on='station_id')

# Forecast 1 year and 3 years ahead
forecast_base['WL_forecast_1y'] = (
    forecast_base['WL_MBGL'] + forecast_base['mean_delta']
).clip(0, 50)

forecast_base['WL_forecast_3y'] = (
    forecast_base['WL_MBGL'] + (3 * forecast_base['mean_delta'])
).clip(0, 50)

print("="*70)
print("BASELINE FORECAST MODEL COMPLETE")
print("="*70)
print(f"\n✓ Forecasts generated for {len(forecast_base)} stations")
print(f"\nCurrent WL statistics:")
print(forecast_base['WL_MBGL'].describe())
print(f"\n1-Year Forecast WL statistics:")
print(forecast_base['WL_forecast_1y'].describe())
print(f"\n3-Year Forecast WL statistics:")
print(forecast_base['WL_forecast_3y'].describe())
print("\n" + "="*70)

print("\nSample forecasts:")
print(forecast_base[['station_id', 'WL_MBGL', 'mean_delta', 'WL_forecast_1y', 'WL_forecast_3y']].head(10))

BASELINE FORECAST MODEL COMPLETE

✓ Forecasts generated for 9545 stations

Current WL statistics:
count    9545.000000
mean        6.570801
std         6.176830
min         0.000000
25%         3.000000
50%         4.800000
75%         7.900000
max        49.800000
Name: WL_MBGL, dtype: float64

1-Year Forecast WL statistics:
count    9545.000000
mean        6.532359
std         6.353908
min         0.000000
25%         2.866667
50%         4.766667
75%         7.888889
max        50.000000
Name: WL_forecast_1y, dtype: float64

3-Year Forecast WL statistics:
count    9545.000000
mean        6.487823
std         6.724253
min         0.000000
25%         2.600000
50%         4.656250
75%         7.914286
max        50.000000
Name: WL_forecast_3y, dtype: float64


Sample forecasts:
                     station_id  WL_MBGL  mean_delta  WL_forecast_1y  \
0             Delhi_nan_77.0083      2.8       -0.30            2.50   
1          Punjab_31.275_76.375      4.9        0.66            5.

## STEP 5.5 — FORECAST → GAVI (CRITICAL INTEGRATION)

Convert forecasted WL to future GAVI using the same formula:
- Merge with baseline (min_wl, max_wl)
- Compute GAVI for forecasted values
- Clip to 0-100 range
- Assign categories

In [46]:
# STEP 5.5: Convert forecasted WL to GAVI

# Merge with baseline
forecast_gavi = forecast_base.merge(
    baseline[['station_id', 'min_wl', 'max_wl']],
    on='station_id',
    how='inner'
)

# Compute GAVI for 1-year forecast
forecast_gavi['GAVI_forecast_1y'] = 100 * (
    1 - (forecast_gavi['WL_forecast_1y'] - forecast_gavi['min_wl']) /
        (forecast_gavi['max_wl'] - forecast_gavi['min_wl'])
)
forecast_gavi['GAVI_forecast_1y'] = forecast_gavi['GAVI_forecast_1y'].clip(0, 100)

# Compute GAVI for 3-year forecast
forecast_gavi['GAVI_forecast_3y'] = 100 * (
    1 - (forecast_gavi['WL_forecast_3y'] - forecast_gavi['min_wl']) /
        (forecast_gavi['max_wl'] - forecast_gavi['min_wl'])
)
forecast_gavi['GAVI_forecast_3y'] = forecast_gavi['GAVI_forecast_3y'].clip(0, 100)

# Apply GAVI category function
forecast_gavi['GAVI_CATEGORY_1y'] = forecast_gavi['GAVI_forecast_1y'].apply(gavi_category)
forecast_gavi['GAVI_CATEGORY_3y'] = forecast_gavi['GAVI_forecast_3y'].apply(gavi_category)

print("="*70)
print("FORECAST → GAVI CONVERSION COMPLETE")
print("="*70)
print(f"\n✓ Future GAVI computed for {len(forecast_gavi)} stations")
print(f"\nCurrent GAVI statistics:")
print(forecast_gavi['GAVI'].describe())
print(f"\n1-Year Forecast GAVI statistics:")
print(forecast_gavi['GAVI_forecast_1y'].describe())
print(f"\n3-Year Forecast GAVI statistics:")
print(forecast_gavi['GAVI_forecast_3y'].describe())
print("\n" + "="*70)

print("\n1-Year Forecast GAVI Categories:")
print(forecast_gavi['GAVI_CATEGORY_1y'].value_counts())
print("\n3-Year Forecast GAVI Categories:")
print(forecast_gavi['GAVI_CATEGORY_3y'].value_counts())

FORECAST → GAVI CONVERSION COMPLETE

✓ Future GAVI computed for 9545 stations

Current GAVI statistics:
count    9545.000000
mean       54.095097
std        35.560261
min         0.000000
25%        22.222222
50%        57.894737
75%        87.500000
max       100.000000
Name: GAVI, dtype: float64

1-Year Forecast GAVI statistics:
count    9545.000000
mean       54.416596
std        37.305828
min         0.000000
25%        18.045113
50%        58.962264
75%        91.927083
max       100.000000
Name: GAVI_forecast_1y, dtype: float64

3-Year Forecast GAVI statistics:
count    9545.000000
mean       54.732805
std        39.794459
min         0.000000
25%        10.000000
50%        60.344828
75%       100.000000
max       100.000000
Name: GAVI_forecast_3y, dtype: float64


1-Year Forecast GAVI Categories:
GAVI_CATEGORY_1y
Safe        3671
Critical    2725
Watch       1709
Stressed    1440
Name: count, dtype: int64

3-Year Forecast GAVI Categories:
GAVI_CATEGORY_3y
Safe        3962
Criti

## STEP 5.6 — FUTURE ALERT LOGIC (WINNER MOVE)

Generate predictive alerts based on forecasted GAVI:
- 🔴 FUTURE_CRITICAL: Forecast GAVI < 25
- 🟠 EARLY_DEPLETION_WARNING: GAVI drops category
- 🟢 RECOVERY_EXPECTED: GAVI improves
- ⚪ STABLE: No significant change

**This is where AI meets policy - anticipation, not just detection.**

In [47]:
# STEP 5.6: Future alert logic

def future_alert_1y(row):
    """
    Generate predictive alerts for 1-year forecast
    """
    if row['GAVI_forecast_1y'] < 25:
        return 'FUTURE_CRITICAL'
    if row['GAVI_forecast_1y'] < row['GAVI'] - 10:  # Significant drop
        return 'EARLY_DEPLETION_WARNING'
    if row['GAVI_forecast_1y'] > row['GAVI'] + 10:  # Significant improvement
        return 'RECOVERY_EXPECTED'
    return 'STABLE'

def future_alert_3y(row):
    """
    Generate predictive alerts for 3-year forecast
    """
    if row['GAVI_forecast_3y'] < 25:
        return 'FUTURE_CRITICAL'
    if row['GAVI_forecast_3y'] < row['GAVI'] - 15:  # Significant drop
        return 'EARLY_DEPLETION_WARNING'
    if row['GAVI_forecast_3y'] > row['GAVI'] + 15:  # Significant improvement
        return 'RECOVERY_EXPECTED'
    return 'STABLE'

# Apply future alert functions
forecast_gavi['FUTURE_ALERT_1y'] = forecast_gavi.apply(future_alert_1y, axis=1)
forecast_gavi['FUTURE_ALERT_3y'] = forecast_gavi.apply(future_alert_3y, axis=1)

print("="*70)
print("PREDICTIVE ALERT SYSTEM COMPLETE")
print("="*70)
print(f"\n✓ Future alerts generated for {len(forecast_gavi)} stations")
print("\n1-Year Future Alert Distribution:")
print(forecast_gavi['FUTURE_ALERT_1y'].value_counts())
print(f"\nPercentage:")
print((forecast_gavi['FUTURE_ALERT_1y'].value_counts(normalize=True) * 100).round(2))
print("\n" + "="*70)
print("\n3-Year Future Alert Distribution:")
print(forecast_gavi['FUTURE_ALERT_3y'].value_counts())
print(f"\nPercentage:")
print((forecast_gavi['FUTURE_ALERT_3y'].value_counts(normalize=True) * 100).round(2))
print("\n" + "="*70)

# Show critical future cases
critical_future = forecast_gavi[forecast_gavi['FUTURE_ALERT_1y'] == 'FUTURE_CRITICAL']
print(f"\n🔴 Stations predicted to be CRITICAL in 1 year: {len(critical_future)}")
if len(critical_future) > 0:
    print("\nTop 10 critical future cases:")
    print(critical_future[['station_id', 'STATE_UT', 'DISTRICT', 'GAVI', 'GAVI_forecast_1y']].head(10))

PREDICTIVE ALERT SYSTEM COMPLETE

✓ Future alerts generated for 9545 stations

1-Year Future Alert Distribution:
FUTURE_ALERT_1y
STABLE                     6568
FUTURE_CRITICAL            2725
RECOVERY_EXPECTED           234
EARLY_DEPLETION_WARNING      18
Name: count, dtype: int64

Percentage:
FUTURE_ALERT_1y
STABLE                     68.81
FUTURE_CRITICAL            28.55
RECOVERY_EXPECTED           2.45
EARLY_DEPLETION_WARNING     0.19
Name: proportion, dtype: float64


3-Year Future Alert Distribution:
FUTURE_ALERT_3y
STABLE                     5412
FUTURE_CRITICAL            2959
RECOVERY_EXPECTED           973
EARLY_DEPLETION_WARNING     201
Name: count, dtype: int64

Percentage:
FUTURE_ALERT_3y
STABLE                     56.70
FUTURE_CRITICAL            31.00
RECOVERY_EXPECTED          10.19
EARLY_DEPLETION_WARNING     2.11
Name: proportion, dtype: float64


🔴 Stations predicted to be CRITICAL in 1 year: 2725

Top 10 critical future cases:
                        station_id    

## STEP 5.7 — VALIDATION (LIGHTWEIGHT, JUDGE-SAFE)

Validate forecasts using historical data:
- Backtest on 2015-2022 → predict 2023-2024
- Compute MAE (Mean Absolute Error)
- Show sample actual vs predicted comparison

In [48]:
# STEP 5.7: Lightweight validation

# Backtest: Use data up to 2022 to predict 2023
train_data = df_model[df_model['year'] <= 2022].copy()
test_data = df_model[df_model['year'] == 2023].copy()

if len(test_data) > 0:
    # Calculate trend from training data
    backtest_trend = (
        train_data.groupby('station_id')['delta_wl']
        .mean()
        .reset_index(name='mean_delta')
    )
    
    # Get 2022 WL for prediction
    wl_2022 = (
        train_data[train_data['year'] == 2022]
        [['station_id', 'WL_MBGL']]
        .reset_index(drop=True)
    )
    
    # Make predictions
    backtest = wl_2022.merge(backtest_trend, on='station_id')
    backtest['WL_predicted_2023'] = (
        backtest['WL_MBGL'] + backtest['mean_delta']
    ).clip(0, 50)
    
    # Get actual 2023 values
    actual_2023 = test_data[['station_id', 'WL_MBGL']].rename(
        columns={'WL_MBGL': 'WL_actual_2023'}
    )
    
    # Merge predictions with actuals
    validation = backtest.merge(actual_2023, on='station_id', how='inner')
    
    # Calculate MAE
    validation['error'] = abs(validation['WL_actual_2023'] - validation['WL_predicted_2023'])
    mae = validation['error'].mean()
    
    print("="*70)
    print("MODEL VALIDATION RESULTS")
    print("="*70)
    print(f"\n✓ Validation performed on {len(validation)} stations")
    print(f"\n📊 Mean Absolute Error (MAE): {mae:.3f} meters")
    print(f"📊 Relative MAE: {(mae / validation['WL_actual_2023'].mean() * 100):.2f}%")
    print("\nError statistics:")
    print(validation['error'].describe())
    print("\n" + "="*70)
    
    print("\nSample predictions vs actuals (first 10 stations):")
    print(validation[['station_id', 'WL_actual_2023', 'WL_predicted_2023', 'error']].head(10))
    
    # Accuracy metrics
    within_1m = (validation['error'] <= 1.0).sum()
    within_2m = (validation['error'] <= 2.0).sum()
    print(f"\n✓ Predictions within 1m: {within_1m} ({within_1m/len(validation)*100:.1f}%)")
    print(f"✓ Predictions within 2m: {within_2m} ({within_2m/len(validation)*100:.1f}%)")
else:
    print("⚠ No 2023 data available for validation")
    mae = None

print("\n" + "="*70)

MODEL VALIDATION RESULTS

✓ Validation performed on 8227 stations

📊 Mean Absolute Error (MAE): 1.631 meters
📊 Relative MAE: 25.07%

Error statistics:
count    8227.000000
mean        1.631459
std         2.131872
min         0.000000
25%         0.400000
50%         0.971429
75%         2.057143
max        35.300000
Name: error, dtype: float64


Sample predictions vs actuals (first 10 stations):
                       station_id  WL_actual_2023  WL_predicted_2023     error
0   Andhra Pradesh_13.1956_79.004             5.1           1.475000  3.625000
1  Andhra Pradesh_13.2075_79.0981             1.7           1.300000  0.400000
2  Andhra Pradesh_13.4262_79.4531             3.8           1.575000  2.225000
3  Andhra Pradesh_13.6371_79.9008             1.5           2.542857  1.042857
4  Andhra Pradesh_13.6663_78.4184             0.9           1.285714  0.385714
5  Andhra Pradesh_13.8275_78.7878             4.3           3.457143  0.842857
6   Andhra Pradesh_13.908_79.7315             0

## STEP 5.8 — SAVE FORECAST RESULTS & FREEZE

Save all forecast datasets and prepare for visualization & API integration.

In [49]:
# STEP 5.8: Save forecast results

print("="*70)
print("SAVING FORECAST DATASETS")
print("="*70)

# 1. Save complete forecast with GAVI and alerts
forecast_file = 'output/groundwater_forecast_gavi_alerts.csv'
forecast_gavi.to_csv(forecast_file, index=False)
print(f"\n✓ Saved: {forecast_file}")
print(f"   Records: {len(forecast_gavi)}")
print(f"   Columns: {list(forecast_gavi.columns)}")

# 2. Save only critical future alerts for immediate action
critical_forecast = forecast_gavi[
    (forecast_gavi['FUTURE_ALERT_1y'] == 'FUTURE_CRITICAL') |
    (forecast_gavi['FUTURE_ALERT_3y'] == 'FUTURE_CRITICAL')
][['station_id', 'STATE_UT', 'DISTRICT', 'GAVI', 'GAVI_forecast_1y', 
   'GAVI_forecast_3y', 'FUTURE_ALERT_1y', 'FUTURE_ALERT_3y']]

critical_forecast_file = 'output/critical_future_alerts.csv'
critical_forecast.to_csv(critical_forecast_file, index=False)
print(f"\n✓ Saved: {critical_forecast_file}")
print(f"   Critical future stations: {len(critical_forecast)}")

# 3. Save district-level future alerts aggregation
district_future_alerts = forecast_gavi.groupby(['STATE_UT', 'DISTRICT']).agg(
    total_stations=('station_id', 'count'),
    future_critical_1y=('FUTURE_ALERT_1y', lambda x: (x == 'FUTURE_CRITICAL').sum()),
    future_critical_3y=('FUTURE_ALERT_3y', lambda x: (x == 'FUTURE_CRITICAL').sum()),
    early_warning_1y=('FUTURE_ALERT_1y', lambda x: (x == 'EARLY_DEPLETION_WARNING').sum()),
    recovery_expected_1y=('FUTURE_ALERT_1y', lambda x: (x == 'RECOVERY_EXPECTED').sum()),
    avg_current_gavi=('GAVI', 'mean'),
    avg_forecast_1y_gavi=('GAVI_forecast_1y', 'mean'),
    avg_forecast_3y_gavi=('GAVI_forecast_3y', 'mean')
).reset_index()

district_future_alerts = district_future_alerts.sort_values('future_critical_1y', ascending=False)

district_future_file = 'output/district_future_alerts.csv'
district_future_alerts.to_csv(district_future_file, index=False)
print(f"\n✓ Saved: {district_future_file}")
print(f"   Districts: {len(district_future_alerts)}")

print("\n" + "="*70)
print("✓ ALL FORECAST DATASETS SAVED SUCCESSFULLY!")
print("="*70)

print("\n📁 Files created:")
print(f"  1. {forecast_file}")
print(f"  2. {critical_forecast_file}")
print(f"  3. {district_future_file}")

print("\n" + "="*70)
print("🎯 SECTION 5 — AI/ANALYTICS COMPLETE AND FROZEN")
print("="*70)
print("\n✅ Forecast logic frozen")
print("✅ Thresholds frozen")
print("✅ Ready for visualization & API")
print("\n🚀 NEXT STEPS:")
print("   → Create visualizations (maps, charts)")
print("   → Build API endpoints")
print("   → Generate presentation materials")

SAVING FORECAST DATASETS

✓ Saved: output/groundwater_forecast_gavi_alerts.csv
   Records: 9545
   Columns: ['station_id', 'year', 'WL_MBGL', 'GAVI', 'STATE_UT', 'DISTRICT', 'mean_delta', 'WL_forecast_1y', 'WL_forecast_3y', 'min_wl', 'max_wl', 'GAVI_forecast_1y', 'GAVI_forecast_3y', 'GAVI_CATEGORY_1y', 'GAVI_CATEGORY_3y', 'FUTURE_ALERT_1y', 'FUTURE_ALERT_3y']

✓ Saved: output/critical_future_alerts.csv
   Critical future stations: 3004

✓ Saved: output/district_future_alerts.csv
   Districts: 713

✓ ALL FORECAST DATASETS SAVED SUCCESSFULLY!

📁 Files created:
  1. output/groundwater_forecast_gavi_alerts.csv
  2. output/critical_future_alerts.csv
  3. output/district_future_alerts.csv

🎯 SECTION 5 — AI/ANALYTICS COMPLETE AND FROZEN

✅ Forecast logic frozen
✅ Thresholds frozen
✅ Ready for visualization & API

🚀 NEXT STEPS:
   → Create visualizations (maps, charts)
   → Build API endpoints
   → Generate presentation materials


## SUMMARY: PREDICTIVE ANALYTICS CAPABILITIES

**What we built:**
1. ✅ **Baseline Forecast Model** - Trend-based prediction using historical delta_wl
2. ✅ **WL → GAVI Conversion** - Future water levels converted to GAVI scores
3. ✅ **Predictive Alerts** - 4 types of future alerts (Critical, Warning, Recovery, Stable)
4. ✅ **Validation** - Lightweight MAE-based validation on historical data
5. ✅ **District Aggregation** - Policy-ready future alerts by district

**Key Metrics:**
- Forecast horizon: 1 year & 3 years
- Forecast unit: Annual (matches data frequency)
- Alert thresholds: GAVI < 25 (Critical), ±10 point change (Warning/Recovery)

**Outputs:**
- `groundwater_forecast_gavi_alerts.csv` - Complete forecast dataset
- `critical_future_alerts.csv` - Stations requiring immediate attention
- `district_future_alerts.csv` - District-level future risk assessment

**Innovation:**
🔥 **This is anticipation, not just detection** - Early warning system for policy intervention before crisis hits.

# SECTION 6 — GEO-SPATIAL & VISUALIZATION DESIGN (WINNING SETUP)

## 🎯 DESIGN PRINCIPLE (VERY IMPORTANT)

**One map answers "WHERE is the problem?"**  
**One chart answers "WHY is it happening?"**  
**One overlay answers "WHAT will happen next?"**

If you follow this, judges instantly get it.

---

## Visual Strategy:

| Visual | Question Answered |
|--------|-------------------|
| District Stress Map | Where is action needed? |
| Station Alert Map | Which wells are failing? |
| Time Series Chart | Why is it happening? |
| Forecast Overlay | What happens next? |

This is perfect storytelling.

In [50]:
# Install required visualization libraries
# Uncomment and run if not already installed

# !pip install folium geopandas plotly kaleido

## 🟦 MAP 1 (FLAGSHIP): District-Level Groundwater Stress Map

🔥 **This is your HERO visualization**

### 📌 What this map shows (in one glance)
- Each district colored by **% of stressed stations**
- **Stressed = stations where GAVI < 50**
- This is policy language, not ML language

### 🎨 COLOR SCALE (LOCKED)

| Color | Meaning | Condition |
|-------|---------|-----------|
| 🟢 Green | Safe | < 20% stressed |
| 🟡 Yellow | Watch | 20–40% |
| 🟠 Orange | Stressed | 40–60% |
| 🔴 Red | Critical | > 60% |

⚠️ **Never use gradients. Discrete colors = clearer decisions.**

### 🧲 TOOLTIP DESIGN (MOST IMPORTANT PART)

When user hovers on a district, show:
```
📍 District: Prayagraj
📊 Avg GAVI: 41 (Stressed)
🚨 Stressed Stations: 58%
🔴 Critical Alerts: 12
🔮 Future Risk (1y): HIGH
```

This turns a map into a decision console.

In [51]:
import folium
from folium import plugins
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("="*70)
print("VISUALIZATION LIBRARIES LOADED")
print("="*70)

VISUALIZATION LIBRARIES LOADED


In [63]:
# STEP 6.1: Prepare district-level visualization data (USING CURRENT SNAPSHOT)

# Use district_stress_current (current snapshot) instead of district_stress (historical)
district_viz = district_stress_current.copy()

# Add stress category based on stressed_ratio
def stress_category(ratio):
    if ratio < 20:
        return "Safe"
    elif ratio < 40:
        return "Watch"
    elif ratio < 60:
        return "Stressed"
    else:
        return "Critical"

district_viz['stress_category'] = district_viz['stressed_ratio'].apply(stress_category)

# Merge with forecast data to add future risk
district_future_risk = district_future_alerts[['STATE_UT', 'DISTRICT', 'future_critical_1y']].copy()
district_viz = district_viz.merge(district_future_risk, on=['STATE_UT', 'DISTRICT'], how='left')

# Add future risk flag
district_viz['future_risk_flag'] = district_viz['future_critical_1y'].apply(
    lambda x: 'HIGH' if x >= 3 else ('MEDIUM' if x >= 1 else 'LOW')
)

# Color mapping for stress categories
color_map = {
    'Safe': '#2ecc71',      # Green
    'Watch': '#f1c40f',     # Yellow
    'Stressed': '#e67e22',  # Orange
    'Critical': '#e74c3c'   # Red
}

district_viz['color'] = district_viz['stress_category'].map(color_map)

print("="*70)
print("DISTRICT VISUALIZATION DATA PREPARED (CURRENT SNAPSHOT)")
print("="*70)
print(f"\n✓ Total districts: {len(district_viz)}")
print(f"\nStress category distribution (CURRENT):")
print(district_viz['stress_category'].value_counts())
print(f"\nFuture risk distribution:")
print(district_viz['future_risk_flag'].value_counts())
print("\n" + "="*70)

# Show top critical districts
critical_districts_viz = district_viz[district_viz['stress_category'] == 'Critical'].sort_values(
    'stressed_ratio', ascending=False
)
print(f"\n🔴 Top 10 CRITICAL districts (CURRENT snapshot):")
print(critical_districts_viz[['STATE_UT', 'DISTRICT', 'stressed_ratio', 'avg_gavi', 'critical_alerts', 'future_risk_flag']].head(10))

DISTRICT VISUALIZATION DATA PREPARED (CURRENT SNAPSHOT)

✓ Total districts: 713

Stress category distribution (CURRENT):
stress_category
Critical    205
Stressed    184
Watch       181
Safe        143
Name: count, dtype: int64

Future risk distribution:
future_risk_flag
HIGH      340
MEDIUM    223
LOW       150
Name: count, dtype: int64


🔴 Top 10 CRITICAL districts (CURRENT snapshot):
        STATE_UT         DISTRICT  stressed_ratio   avg_gavi  critical_alerts  \
0  Uttar Pradesh       Mahrajganj           100.0  31.641124                1   
1  Uttar Pradesh           Rampur           100.0   6.818182                1   
2          Bihar       Sheikhpura           100.0   9.438776                2   
3          Bihar        Sitamarhi           100.0   8.056634                3   
4          Bihar          Sheohar           100.0   0.000000                1   
5        Tripura     West Tripura           100.0  32.367632                1   
6          Bihar            Siwan           

In [53]:
# STEP 6.2: Create District Stress Bar Chart (Plotly)
# This is cleaner than maps for showing exact rankings

# Sort by stress ratio
district_viz_sorted = district_viz.sort_values('stressed_ratio', ascending=True).tail(30)  # Top 30

fig_district_bar = go.Figure()

fig_district_bar.add_trace(go.Bar(
    y=district_viz_sorted['DISTRICT'] + ', ' + district_viz_sorted['STATE_UT'],
    x=district_viz_sorted['stressed_ratio'],
    orientation='h',
    marker=dict(
        color=district_viz_sorted['stressed_ratio'],
        colorscale=[
            [0, '#2ecc71'],      # Green
            [0.2, '#f1c40f'],    # Yellow
            [0.4, '#e67e22'],    # Orange
            [1.0, '#e74c3c']     # Red
        ],
        showscale=True,
        colorbar=dict(title="Stressed %")
    ),
    text=district_viz_sorted['stressed_ratio'].round(1).astype(str) + '%',
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>' +
                  'Stressed Stations: %{x:.1f}%<br>' +
                  '<extra></extra>'
))

fig_district_bar.update_layout(
    title={
        'text': '🗺️ Top 30 Most Stressed Districts (by % of Stressed Stations)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial Black'}
    },
    xaxis_title="% of Stations with GAVI < 50",
    yaxis_title="District",
    height=900,
    template='plotly_white',
    showlegend=False,
    margin=dict(l=250, r=100, t=80, b=80)
)

fig_district_bar.add_vline(x=20, line_dash="dash", line_color="green", opacity=0.5, annotation_text="Safe")
fig_district_bar.add_vline(x=40, line_dash="dash", line_color="yellow", opacity=0.5, annotation_text="Watch")
fig_district_bar.add_vline(x=60, line_dash="dash", line_color="orange", opacity=0.5, annotation_text="Stressed")

# Save chart
fig_district_bar.write_html('output/district_stress_chart.html')
print("✓ District stress chart saved: output/district_stress_chart.html")

# Show chart (commented out - open the HTML file in browser instead)
# fig_district_bar.show()

✓ District stress chart saved: output/district_stress_chart.html


## 🟦 MAP 2: Station-Level Alert Map

### What this map shows
- Each DWLR station = a dot
- Dot color = current alert

| Color | Alert |
|-------|-------|
| 🔴 Red | CRITICAL_GROUNDWATER |
| 🟠 Orange | DEPLETION_WARNING |
| 🟡 Yellow | SUDDEN_DROP |
| 🟢 Green | RECOVERY_SIGNAL |
| ⚪ White | NORMAL |

📌 **Use marker clustering — essential.**

### Tooltip for station:
```
Station ID: UP_134
GAVI: 23 (Critical)
ΔWL: -2.4 m
Alert: CRITICAL_GROUNDWATER
```

This map proves granularity & coverage.

In [54]:
# STEP 6.3: Prepare station-level map data

# Get latest observation per station with alerts
station_map_data = (
    df_gavi.sort_values('DATE')
    .groupby('station_id')
    .tail(1)
    .reset_index(drop=True)
)

# Filter stations with valid coordinates
station_map_data = station_map_data[
    (station_map_data['LATITUDE'].notna()) & 
    (station_map_data['LONGITUDE'].notna()) &
    (station_map_data['LATITUDE'] != 0) &
    (station_map_data['LONGITUDE'] != 0)
].copy()

# Alert color mapping
alert_color_map = {
    'CRITICAL_GROUNDWATER': '#e74c3c',    # Red
    'DEPLETION_WARNING': '#e67e22',       # Orange
    'SUDDEN_DROP': '#f1c40f',             # Yellow
    'RECOVERY_SIGNAL': '#2ecc71',         # Green
    'NORMAL': '#95a5a6'                   # Gray
}

station_map_data['alert_color'] = station_map_data['ALERT_CONFIRMED'].map(alert_color_map)

print("="*70)
print("STATION MAP DATA PREPARED")
print("="*70)
print(f"\n✓ Total stations with coordinates: {len(station_map_data)}")
print(f"\nAlert distribution:")
print(station_map_data['ALERT_CONFIRMED'].value_counts())
print("\n" + "="*70)

# Show sample data
print("\nSample station data:")
print(station_map_data[['station_id', 'STATE_UT', 'DISTRICT', 'LATITUDE', 'LONGITUDE', 
                         'GAVI', 'ALERT_CONFIRMED', 'alert_color']].head(10))

STATION MAP DATA PREPARED

✓ Total stations with coordinates: 9339

Alert distribution:
ALERT_CONFIRMED
NORMAL                  5480
RECOVERY_SIGNAL         1198
CRITICAL_GROUNDWATER    1097
SUDDEN_DROP             1043
DEPLETION_WARNING        521
Name: count, dtype: int64


Sample station data:
                           station_id           STATE_UT  \
0           Telangana_18.2333_80.2542          Telangana   
1               Delhi_28.5767_76.9142              Delhi   
3               Delhi_28.7433_77.2214              Delhi   
4             Telangana_17.15_77.7194          Telangana   
5   Arunachal Pradesh_27.3083_93.9694  Arunachal Pradesh   
6               Assam_24.7275_92.5833              Assam   
7              Telangana_18.8_79.9167          Telangana   
8               Assam_26.1111_90.5486              Assam   
9           Meghalaya_25.5928_91.9192          Meghalaya   
10              Assam_27.7028_94.8558              Assam   

                    DISTRICT  LATITUDE  L

In [55]:
# STEP 6.4: Create Station Alert Map with Folium

# Create base map centered on India
map_center = [station_map_data['LATITUDE'].mean(), station_map_data['LONGITUDE'].mean()]
station_map = folium.Map(
    location=map_center,
    zoom_start=5,
    tiles='OpenStreetMap'
)

# Create marker cluster
marker_cluster = plugins.MarkerCluster(name='Station Alerts').add_to(station_map)

# Add markers for each station
for idx, row in station_map_data.iterrows():
    # Create popup content
    popup_html = f"""
    <div style="font-family: Arial; width: 250px;">
        <h4 style="margin: 0; color: #2c3e50;">Station: {row['station_id']}</h4>
        <hr style="margin: 5px 0;">
        <b>📍 Location:</b> {row['DISTRICT']}, {row['STATE_UT']}<br>
        <b>📊 GAVI:</b> {row['GAVI']:.1f} ({row['GAVI_CATEGORY']})<br>
        <b>💧 WL:</b> {row['WL_MBGL']:.2f} mbgl<br>
        <b>📉 ΔWL:</b> {row['delta_wl']:.2f} m<br>
        <b>🚨 Alert:</b> <span style="color: {row['alert_color']}; font-weight: bold;">{row['ALERT_CONFIRMED']}</span><br>
        <b>📋 Action:</b> {row['SUGGESTED_ACTION'][:50]}...
    </div>
    """
    
    # Determine icon color based on alert
    icon_color_map = {
        'CRITICAL_GROUNDWATER': 'red',
        'DEPLETION_WARNING': 'orange',
        'SUDDEN_DROP': 'yellow',
        'RECOVERY_SIGNAL': 'green',
        'NORMAL': 'gray'
    }
    
    icon_color = icon_color_map.get(row['ALERT_CONFIRMED'], 'gray')
    
    # Add marker
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['station_id']} - {row['ALERT_CONFIRMED']}",
        icon=folium.Icon(color=icon_color, icon='tint', prefix='fa')
    ).add_to(marker_cluster)

# Add layer control
folium.LayerControl().add_to(station_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: 180px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px; border-radius: 5px;">
    <p style="margin: 0; font-weight: bold; text-align: center;">Alert Legend</p>
    <hr style="margin: 5px 0;">
    <p style="margin: 3px;"><span style="color: red;">●</span> Critical Groundwater</p>
    <p style="margin: 3px;"><span style="color: orange;">●</span> Depletion Warning</p>
    <p style="margin: 3px;"><span style="color: gold;">●</span> Sudden Drop</p>
    <p style="margin: 3px;"><span style="color: green;">●</span> Recovery Signal</p>
    <p style="margin: 3px;"><span style="color: gray;">●</span> Normal</p>
</div>
'''
station_map.get_root().html.add_child(folium.Element(legend_html))

# Save map
station_map.save('output/station_alert_map.html')
print("="*70)
print("✓ Station alert map saved: output/station_alert_map.html")
print("="*70)
print("\n📍 Map features:")
print(f"   • {len(station_map_data)} stations plotted")
print(f"   • Marker clustering enabled")
print(f"   • Color-coded by alert type")
print(f"   • Interactive tooltips & popups")
print("\n🖱️ Open the HTML file in a browser to interact with the map")

# Display map (commented out - open the HTML file in browser instead)
# station_map

C:\Users\lsing\AppData\Local\Temp\ipykernel_32796\1054592786.py:46: UserWarning:

color argument of Icon should be one of: {'lightblue', 'darkpurple', 'cadetblue', 'gray', 'blue', 'lightgray', 'darkgreen', 'purple', 'green', 'pink', 'lightgreen', 'red', 'orange', 'darkblue', 'lightred', 'beige', 'darkred', 'black', 'white'}.



✓ Station alert map saved: output/station_alert_map.html

📍 Map features:
   • 9339 stations plotted
   • Marker clustering enabled
   • Color-coded by alert type
   • Interactive tooltips & popups

🖱️ Open the HTML file in a browser to interact with the map


## 📈 CHART 1: Station Time-Series (Explain "WHY")

### Single chart, 3 layers:
For one selected station:
1. **WL history** (2015–2024)
2. **GAVI history** (secondary axis)
3. **Forecast** (dashed line)

Judges immediately see: *"Okay, decline → alert → future risk"*

In [56]:
# STEP 6.5: Create Time Series Visualization for Critical Stations

# Select top 3 critical stations for visualization
critical_stations = station_map_data[
    station_map_data['ALERT_CONFIRMED'] == 'CRITICAL_GROUNDWATER'
].sort_values('GAVI').head(3)['station_id'].tolist()

# If no critical stations, select stations with lowest GAVI
if len(critical_stations) == 0:
    critical_stations = station_map_data.sort_values('GAVI').head(3)['station_id'].tolist()

print("="*70)
print("CREATING TIME SERIES CHARTS FOR CRITICAL STATIONS")
print("="*70)
print(f"\nSelected stations: {critical_stations}")
print("\n" + "="*70)

# Function to create time series chart for a station
def create_station_timeseries(station_id):
    # Get historical data for station
    station_history = df_gavi[df_gavi['station_id'] == station_id].sort_values('year').copy()
    
    # Get forecast data for station
    station_forecast = forecast_gavi[forecast_gavi['station_id'] == station_id].copy()
    
    if len(station_history) == 0:
        print(f"⚠ No data found for station {station_id}")
        return None
    
    # Create subplot with secondary y-axis
    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{"secondary_y": True}]],
        subplot_titles=[f"Station {station_id} - Historical & Forecast Analysis"]
    )
    
    # Add WL trace (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=station_history['year'],
            y=station_history['WL_MBGL'],
            name='Water Level (mbgl)',
            mode='lines+markers',
            line=dict(color='#3498db', width=2),
            marker=dict(size=6)
        ),
        secondary_y=False
    )
    
    # Add GAVI trace (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=station_history['year'],
            y=station_history['GAVI'],
            name='GAVI Score',
            mode='lines+markers',
            line=dict(color='#e74c3c', width=2, dash='dot'),
            marker=dict(size=6, symbol='diamond')
        ),
        secondary_y=True
    )
    
    # Add forecast points if available
    if len(station_forecast) > 0:
        last_year = station_history['year'].max()
        
        # 1-year forecast
        fig.add_trace(
            go.Scatter(
                x=[last_year, last_year + 1],
                y=[station_history['WL_MBGL'].iloc[-1], station_forecast['WL_forecast_1y'].iloc[0]],
                name='WL Forecast (1y)',
                mode='lines+markers',
                line=dict(color='#3498db', width=2, dash='dash'),
                marker=dict(size=8, symbol='star')
            ),
            secondary_y=False
        )
        
        # 1-year GAVI forecast
        fig.add_trace(
            go.Scatter(
                x=[last_year, last_year + 1],
                y=[station_history['GAVI'].iloc[-1], station_forecast['GAVI_forecast_1y'].iloc[0]],
                name='GAVI Forecast (1y)',
                mode='lines+markers',
                line=dict(color='#e74c3c', width=2, dash='dash'),
                marker=dict(size=8, symbol='star')
            ),
            secondary_y=True
        )
        
        # 3-year forecast
        fig.add_trace(
            go.Scatter(
                x=[last_year, last_year + 3],
                y=[station_history['WL_MBGL'].iloc[-1], station_forecast['WL_forecast_3y'].iloc[0]],
                name='WL Forecast (3y)',
                mode='lines+markers',
                line=dict(color='#3498db', width=1, dash='dot'),
                marker=dict(size=6, symbol='x'),
                opacity=0.7
            ),
            secondary_y=False
        )
        
        # 3-year GAVI forecast
        fig.add_trace(
            go.Scatter(
                x=[last_year, last_year + 3],
                y=[station_history['GAVI'].iloc[-1], station_forecast['GAVI_forecast_3y'].iloc[0]],
                name='GAVI Forecast (3y)',
                mode='lines+markers',
                line=dict(color='#e74c3c', width=1, dash='dot'),
                marker=dict(size=6, symbol='x'),
                opacity=0.7
            ),
            secondary_y=True
        )
    
    # Add GAVI threshold lines
    fig.add_hline(y=25, line_dash="dash", line_color="red", opacity=0.3, 
                  annotation_text="Critical (GAVI=25)", secondary_y=True)
    fig.add_hline(y=50, line_dash="dash", line_color="orange", opacity=0.3, 
                  annotation_text="Stressed (GAVI=50)", secondary_y=True)
    fig.add_hline(y=75, line_dash="dash", line_color="green", opacity=0.3, 
                  annotation_text="Safe (GAVI=75)", secondary_y=True)
    
    # Update axes
    fig.update_xaxes(title_text="Year", showgrid=True)
    fig.update_yaxes(title_text="Water Level (mbgl)", secondary_y=False, showgrid=True)
    fig.update_yaxes(title_text="GAVI Score", secondary_y=True, range=[0, 100])
    
    # Update layout
    station_info = station_history.iloc[-1]
    fig.update_layout(
        title={
            'text': f"<b>{station_id}</b><br>{station_info['DISTRICT']}, {station_info['STATE_UT']}",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        height=500,
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

# Create charts for selected stations
for i, station_id in enumerate(critical_stations, 1):
    print(f"\n📊 Creating chart {i}/3 for station: {station_id}")
    fig = create_station_timeseries(station_id)
    if fig is not None:
        # Save chart
        output_file = f'output/station_timeseries_{station_id}.html'
        fig.write_html(output_file)
        print(f"   ✓ Saved: {output_file}")
        # Show chart (commented out - open the HTML file in browser instead)
        # fig.show()

print("\n" + "="*70)
print("✓ TIME SERIES CHARTS COMPLETE")
print("="*70)

CREATING TIME SERIES CHARTS FOR CRITICAL STATIONS

Selected stations: ['Maharashtra_19.8167_76.8667', 'Gujarat_22.4208_73.4194', 'Odisha_19.3681_85.0028']


📊 Creating chart 1/3 for station: Maharashtra_19.8167_76.8667
   ✓ Saved: output/station_timeseries_Maharashtra_19.8167_76.8667.html

📊 Creating chart 2/3 for station: Gujarat_22.4208_73.4194
   ✓ Saved: output/station_timeseries_Gujarat_22.4208_73.4194.html

📊 Creating chart 3/3 for station: Odisha_19.3681_85.0028
   ✓ Saved: output/station_timeseries_Odisha_19.3681_85.0028.html

✓ TIME SERIES CHARTS COMPLETE


## 🔮 OVERLAY: Future Risk Highlight

On the district map, add icons for future risk:

| Icon | Meaning |
|------|---------|
| 🔮 | FUTURE_CRITICAL |
| ⚠️ | EARLY_DEPLETION |
| ✅ | RECOVERY_EXPECTED |

This lets you say: *"This map shows not just current stress, but future risk hotspots."*

**That line wins points.**

In [57]:
# STEP 6.6: Create Comprehensive Dashboard Chart with Future Overlay

# Create state-level summary with future risk
state_dashboard = df_gavi.groupby('STATE_UT').agg(
    total_stations=('station_id', 'nunique'),
    avg_gavi=('GAVI', 'mean'),
    critical_alerts=('ALERT_CONFIRMED', lambda x: (x == 'CRITICAL_GROUNDWATER').sum()),
    stressed_pct=('is_stressed_station', lambda x: (x.groupby(df_gavi.loc[x.index, 'station_id']).first().mean() * 100))
).reset_index()

# Merge with future forecast data
state_future = forecast_gavi.groupby('STATE_UT').agg(
    future_critical_1y=('FUTURE_ALERT_1y', lambda x: (x == 'FUTURE_CRITICAL').sum()),
    avg_gavi_forecast_1y=('GAVI_forecast_1y', 'mean')
).reset_index()

state_dashboard = state_dashboard.merge(state_future, on='STATE_UT', how='left')
state_dashboard = state_dashboard.sort_values('stressed_pct', ascending=False)

# Create combined visualization
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '📊 Current Stress by State (% Stressed Stations)',
        '🔮 Future Critical Alerts (1-Year Forecast)',
        '📈 Current vs Future GAVI (Top 10 States)',
        '🚨 Alert Distribution'
    ),
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "pie"}]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.15
)

# Chart 1: Current Stress by State
top_10_states = state_dashboard.head(10)
fig_dashboard.add_trace(
    go.Bar(
        x=top_10_states['STATE_UT'],
        y=top_10_states['stressed_pct'],
        marker=dict(
            color=top_10_states['stressed_pct'],
            colorscale='Reds',
            showscale=False
        ),
        text=top_10_states['stressed_pct'].round(1).astype(str) + '%',
        textposition='outside',
        name='Stressed %',
        showlegend=False
    ),
    row=1, col=1
)

# Chart 2: Future Critical Alerts
fig_dashboard.add_trace(
    go.Bar(
        x=top_10_states['STATE_UT'],
        y=top_10_states['future_critical_1y'],
        marker=dict(color='#e74c3c'),
        text=top_10_states['future_critical_1y'],
        textposition='outside',
        name='Future Critical',
        showlegend=False
    ),
    row=1, col=2
)

# Chart 3: Current vs Future GAVI
fig_dashboard.add_trace(
    go.Scatter(
        x=top_10_states['avg_gavi'],
        y=top_10_states['avg_gavi_forecast_1y'],
        mode='markers+text',
        marker=dict(
            size=top_10_states['total_stations'],
            color=top_10_states['stressed_pct'],
            colorscale='RdYlGn_r',
            showscale=True,
            colorbar=dict(title="Stressed %", x=1.15),
            line=dict(width=1, color='white')
        ),
        text=top_10_states['STATE_UT'],
        textposition='top center',
        name='States',
        showlegend=False
    ),
    row=2, col=1
)

# Add diagonal line (no change line)
fig_dashboard.add_trace(
    go.Scatter(
        x=[0, 100],
        y=[0, 100],
        mode='lines',
        line=dict(dash='dash', color='gray'),
        name='No Change',
        showlegend=False
    ),
    row=2, col=1
)

# Chart 4: Alert Distribution
alert_distribution = df_gavi['ALERT_CONFIRMED'].value_counts()
fig_dashboard.add_trace(
    go.Pie(
        labels=alert_distribution.index,
        values=alert_distribution.values,
        marker=dict(colors=['#95a5a6', '#e74c3c', '#e67e22', '#2ecc71', '#f1c40f']),
        textinfo='label+percent',
        name='Alerts'
    ),
    row=2, col=2
)

# Update layout
fig_dashboard.update_xaxes(title_text="State", row=1, col=1, tickangle=-45)
fig_dashboard.update_xaxes(title_text="State", row=1, col=2, tickangle=-45)
fig_dashboard.update_xaxes(title_text="Current GAVI", row=2, col=1, range=[0, 100])
fig_dashboard.update_yaxes(title_text="% Stressed", row=1, col=1)
fig_dashboard.update_yaxes(title_text="# Critical Alerts", row=1, col=2)
fig_dashboard.update_yaxes(title_text="Forecast GAVI (1y)", row=2, col=1, range=[0, 100])

fig_dashboard.update_layout(
    title={
        'text': '<b>🗺️ NATIONAL GROUNDWATER STRESS DASHBOARD - Current & Future Risk</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'family': 'Arial Black'}
    },
    height=900,
    showlegend=False,
    template='plotly_white'
)

# Save dashboard
fig_dashboard.write_html('output/groundwater_dashboard.html')
print("="*70)
print("✓ Comprehensive dashboard saved: output/groundwater_dashboard.html")
print("="*70)

# Show dashboard (commented out - open the HTML file in browser instead)
# fig_dashboard.show()

✓ Comprehensive dashboard saved: output/groundwater_dashboard.html


## 🧩 MAP → DATA → DECISION (CLEAR FLOW)

| Visual | Question Answered |
|--------|-------------------|
| District Stress Map | Where is action needed? |
| Station Alert Map | Which wells are failing? |
| Time Series Chart | Why is it happening? |
| Forecast Overlay | What happens next? |

**This is perfect storytelling.**

---

## 🧪 DEMO FLOW (SCRIPT THIS)

1. Open district stress map
2. Point to a red district
3. Hover → show stats
4. Click → open station chart
5. Show forecast dip
6. Say: *"This district needs intervention within 1 year."*

**Done. Judges are convinced.**

In [ ]:
# STEP 6.7: Create Alert Summary Table for Presentation

print("="*70)
print("CREATING PRESENTATION-READY ALERT SUMMARY")
print("="*70)

# Top 15 most critical districts
critical_presentation = district_viz.sort_values('stressed_ratio', ascending=False).head(15)[
    ['STATE_UT', 'DISTRICT', 'stressed_ratio', 'avg_gavi', 'critical_alerts', 
     'depletion_alerts', 'future_risk_flag', 'stress_category']
].copy()

# Rename columns for clarity
critical_presentation.columns = [
    'State', 'District', 'Stressed %', 'Avg GAVI', 'Critical Alerts',
    'Depletion Warnings', 'Future Risk', 'Status'
]

# Round numeric values
critical_presentation['Stressed %'] = critical_presentation['Stressed %'].round(1)
critical_presentation['Avg GAVI'] = critical_presentation['Avg GAVI'].round(1)

# Create Plotly table
fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=list(critical_presentation.columns),
        fill_color='#2c3e50',
        font=dict(color='white', size=12, family='Arial Black'),
        align='center',
        height=35
    ),
    cells=dict(
        values=[critical_presentation[col] for col in critical_presentation.columns],
        fill_color=[
            ['#ecf0f1' if i % 2 == 0 else 'white' for i in range(len(critical_presentation))]
        ],
        font=dict(color='#2c3e50', size=11),
        align=['left', 'left', 'center', 'center', 'center', 'center', 'center', 'center'],
        height=30
    )
)])

fig_table.update_layout(
    title={
        'text': '<b>🚨 TOP 15 CRITICAL DISTRICTS - IMMEDIATE ACTION REQUIRED</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial Black'}
    },
    height=600,
    margin=dict(l=20, r=20, t=80, b=20)
)

# Save table
fig_table.write_html('output/critical_districts_table.html')
print("\n✓ Critical districts table saved: output/critical_districts_table.html")

# Show table (commented out - open the HTML file in browser instead)
# fig_table.show()

print("\n" + "="*70)
print("PRESENTATION SUMMARY STATISTICS")
print("="*70)

# Overall statistics (using CURRENT snapshot)
total_stations = latest_station_snapshot['station_id'].nunique()
stressed_stations = (
    latest_station_snapshot[latest_station_snapshot['GAVI'] < 50]
    ['station_id']
    .nunique()
)
critical_now = (latest_station_snapshot['ALERT_CONFIRMED'] == 'CRITICAL_GROUNDWATER').sum()
future_critical_1y = (forecast_gavi['FUTURE_ALERT_1y'] == 'FUTURE_CRITICAL').sum()

print(f"""
📊 NATIONAL GROUNDWATER ASSESSMENT SUMMARY

CURRENT STATUS:
  • Total Monitoring Stations: {total_stations:,}
  • Stations Under Stress (GAVI < 50): {stressed_stations:,} ({stressed_stations/total_stations*100:.1f}%)
  • Active Critical Alerts: {critical_now:,}
  • Average National GAVI: {latest_station_snapshot['GAVI'].mean():.1f}

FUTURE RISK (1-YEAR FORECAST):
  • Stations Predicted Critical: {future_critical_1y}
  • Average Forecast GAVI: {forecast_gavi['GAVI_forecast_1y'].mean():.1f}
  • Districts at HIGH Risk: {(district_viz['future_risk_flag'] == 'HIGH').sum()}

TOP 3 MOST CRITICAL DISTRICTS:
""")

for idx, row in critical_presentation.head(3).iterrows():
    print(f"  {idx+1}. {row['District']}, {row['State']}")
    print(f"     → {row['Stressed %']:.1f}% stations stressed, GAVI={row['Avg GAVI']:.1f}")

    print(f"     → {row['Critical Alerts']} critical alerts, Future Risk: {row['Future Risk']}")
    print("="*70)

    print()

CREATING PRESENTATION-READY ALERT SUMMARY

✓ Critical districts table saved: output/critical_districts_table.html

PRESENTATION SUMMARY STATISTICS

📊 NATIONAL GROUNDWATER ASSESSMENT SUMMARY

CURRENT STATUS:
  • Total Monitoring Stations: 9,545
  • Stations Under Stress (GAVI < 50): 4,133 (43.3%)
  • Active Critical Alerts: 1,109
  • Average National GAVI: 54.1

FUTURE RISK (1-YEAR FORECAST):
  • Stations Predicted Critical: 2725
  • Average Forecast GAVI: 54.4
  • Districts at HIGH Risk: 340

TOP 3 MOST CRITICAL DISTRICTS:

  8. DharashivKalamb, Maharashtra
     → 100.0% stations stressed, GAVI=47.4
     → 0 critical alerts, Future Risk: LOW

  7. Siwan, Bihar
     → 100.0% stations stressed, GAVI=15.4
     → 2 critical alerts, Future Risk: HIGH

  6. West Tripura, Tripura
     → 100.0% stations stressed, GAVI=32.4
     → 1 critical alerts, Future Risk: LOW



: 

In [59]:
# STEP 6.8: Final Visualization Summary & Export

print("="*70)
print("📁 VISUALIZATION OUTPUTS SUMMARY")
print("="*70)

visualization_files = [
    "output/district_stress_chart.html",
    "output/station_alert_map.html",
    "output/groundwater_dashboard.html",
    "output/critical_districts_table.html"
]

# Add station timeseries files
for station_id in critical_stations:
    visualization_files.append(f"output/station_timeseries_{station_id}.html")

print("\n✅ ALL VISUALIZATIONS CREATED:")
for i, file in enumerate(visualization_files, 1):
    print(f"  {i}. {file}")

print("\n" + "="*70)
print("🎯 VISUALIZATION SYSTEM COMPLETE")
print("="*70)

print("""
📊 WHAT YOU HAVE NOW:

1. 🗺️  HERO MAP: District stress bar chart (Top 30)
2. 📍 STATION MAP: Interactive Folium map with clustering
3. 📈 TIME SERIES: Historical + forecast for critical stations
4. 🔮 DASHBOARD: 4-panel comprehensive overview
5. 📋 TABLE: Top 15 critical districts summary

🎬 DEMO SCRIPT:

Step 1: Open district_stress_chart.html
        → Point to red bars (>60% stressed)
        → "These districts need immediate intervention"

Step 2: Open station_alert_map.html
        → Zoom into critical district
        → Click on red markers
        → "These are the failing wells"

Step 3: Open station_timeseries_[ID].html
        → Show declining trend
        → Point to forecast dip
        → "This station will reach critical in 1 year"

Step 4: Open groundwater_dashboard.html
        → Show all 4 panels simultaneously
        → "Current stress + Future risk in one view"

Step 5: Open critical_districts_table.html
        → "15 districts requiring immediate policy action"

🏆 WINNING STATEMENT:

"Our system doesn't just detect problems—it predicts them.
 This is anticipation, not reaction.
 Policy makers can act BEFORE the crisis hits."

✨ That's how you win a hackathon.
""")

print("="*70)
print("🚀 SECTION 6 — GEO-SPATIAL & VISUALIZATION COMPLETE")
print("="*70)
print("\n✅ All maps created")
print("✅ All charts created")
print("✅ Dashboard ready")
print("✅ Demo script prepared")
print("\n🎯 NEXT STEPS:")
print("   → Review visualizations in browser")
print("   → Practice demo flow")
print("   → Prepare presentation slides")
print("   → DOMINATE THE HACKATHON! 🏆")
print("\n" + "="*70)

📁 VISUALIZATION OUTPUTS SUMMARY

✅ ALL VISUALIZATIONS CREATED:
  1. output/district_stress_chart.html
  2. output/station_alert_map.html
  3. output/groundwater_dashboard.html
  4. output/critical_districts_table.html
  5. output/station_timeseries_Maharashtra_19.8167_76.8667.html
  6. output/station_timeseries_Gujarat_22.4208_73.4194.html
  7. output/station_timeseries_Odisha_19.3681_85.0028.html

🎯 VISUALIZATION SYSTEM COMPLETE

📊 WHAT YOU HAVE NOW:

1. 🗺️  HERO MAP: District stress bar chart (Top 30)
2. 📍 STATION MAP: Interactive Folium map with clustering
3. 📈 TIME SERIES: Historical + forecast for critical stations
4. 🔮 DASHBOARD: 4-panel comprehensive overview
5. 📋 TABLE: Top 15 critical districts summary

🎬 DEMO SCRIPT:

Step 1: Open district_stress_chart.html
        → Point to red bars (>60% stressed)
        → "These districts need immediate intervention"

Step 2: Open station_alert_map.html
        → Zoom into critical district
        → Click on red markers
        → "These